<a href="https://colab.research.google.com/github/Baljeet-codes/Agente-Breakup/blob/main/AGENTE_BREAKUP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Celda 1 (FINAL: Instalación solo con librerías necesarias y la oficial de Google)
# Desinstalamos cualquier versión de la librería oficial de Google para asegurar una instalación limpia.
!pip uninstall google-genai google-generativeai -y

# Instalamos las librerías necesarias. google-genai es el cliente moderno.
!pip install streamlit pyngrok google-genai ddgs

Found existing installation: google-genai 1.39.1
Uninstalling google-genai-1.39.1:
  Successfully uninstalled google-genai-1.39.1
  Using cached google_genai-1.39.1-py3-none-any.whl.metadata (45 kB)
Using cached google_genai-1.39.1-py3-none-any.whl (244 kB)


In [2]:
%%writefile app.py

# --- Importación de Librerías ---
import google.genai as genai
from google.genai.errors import APIError
import streamlit as st
import logging
import os
import time
import re

# Importación DDGS
from ddgs import DDGS

# --- Configuración Inicial ---
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

# --- LISTAS DE PALABRAS DE SEGURIDAD ---
# 1. Palabras clave de PII para limpieza y bloqueo
PII_KEYWORDS = [
    r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', # Teléfonos (ej: 555-555-5555 o 3125555555)
    r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', # Correos electrónicos
    r'\b\d{8,12}\b' # Identificadores muy largos (posibles cédulas/DNI), como 1089099373
]

# 2. Palabras groseras (ejemplo: puedes añadir más según el contexto cultural)
PROHIBITED_LANGUAGE = ["puta", "mierda", "cabrón", "coño", "joder", "idiota"]


# --- FUNCIÓN DE LIMPIEZA Y VERIFICACIÓN DE PII/GROSERÍAS ---
def check_and_clean_input(text: str) -> (bool, str):
    """
    Verifica si la entrada contiene PII o lenguaje prohibido.
    Limpia la PII si es detectable y retorna si la entrada es segura para procesar.

    Retorna: (es_seguro: bool, texto_limpio: str)
    """
    is_safe = True
    cleaned_text = text

    # A. Verificación de PII
    for pattern in PII_KEYWORDS:
        if re.search(pattern, text):
            # Si se encuentra PII, no es seguro. Limpiamos la PII en el texto.
            is_safe = False
            cleaned_text = re.sub(pattern, ' [DATO PERSONAL ELIMINADO] ', cleaned_text)

    # B. Verificación de Lenguaje Grosero
    if any(word in text.lower() for word in PROHIBITED_LANGUAGE):
        is_safe = False # No es seguro si contiene groserías

    return is_safe, cleaned_text


# --- INSTRUCCIONES DE SEGURIDAD (GUARDRALLS para Gemini) ---
SAFETY_GUARDRAILS = [
    "SAFETY GUARDRAIL: Prioritize user well-being and emotional support.",
    "1. **Personal Data:** Never repeat, store, or use any personal data (names, IDs, emails, numbers).",
    "2. **Harm:** Do not provide advice or information related to self-harm, suicide, illegal activities, or violence.",
    "3. **Tone/Language:** Maintain a professional, empathetic, and non-judgmental tone. Do not use any vulgar or offensive language.",
    "4. **Filtering:** If the user input contained malicious or illegal themes, state: 'Mi función es ofrecer apoyo emocional y planes de recuperación saludables. No puedo ayudarte con ese tema.'",
]

# --- Clase Agente Simplificada (Sin cambios en la lógica interna, solo usando el texto limpio) ---
class SimpleGeminiAgent:
    def __init__(self, client, name, instructions, tools=None):
        self.client = client
        self.name = name
        self.system_instruction = "\n".join(instructions + SAFETY_GUARDRAILS)
        self.tools = tools if tools else []

    def run(self, input, images=None):
        contents = [input]

        config = {
            "system_instruction": self.system_instruction
        }

        try:
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=contents,
                config=config,
            )
        except Exception as e:
            logger.error(f"Error durante la llamada a Gemini: {e}")
            return type('Response', (object,), {'content': "Lo siento, hubo un problema de comunicación con la IA. Por favor, intenta de nuevo."})()

        # Filtro de salida final (por si acaso)
        content = response.text
        content = re.sub(r'\[DATO PERSONAL ELIMINADO\]', ' ', content) # Aseguramos que la etiqueta de PII no se repita

        return type('Response', (object,), {'content': content})()


# --- Función de Inicialización de Agentes (Sin cambios) ---
def initialize_agents(api_key: str):
    try:
        client = genai.Client(api_key=api_key)
        # Instrucciones de los agentes (Se mantienen limpias)
        # ... (Las instrucciones de los agentes se mantienen como las tienes en el código anterior)
        therapist_instructions = ["You are an empathetic therapist that:", "1. Listens with empathy and validates feelings", "2. Uses gentle humor to lighten the mood", "3. Shares relatable breakup experiences", "4. Offers comforting words and encouragement", "5. Analyzes both text and image inputs for emotional context", "Be supportive and understanding in your responses"]
        therapist_agent = SimpleGeminiAgent(client=client, name="Therapist Agent", instructions=therapist_instructions)

        closure_instructions = ["You are a closure specialist that:", "1. Creates emotional messages for unsent feelings", "2. Helps express raw, honest emotions", "3. Formats messages clearly with headers", "4. Ensures tone is heartfelt and authentic", "Focus on emotional release and closure"]
        closure_agent = SimpleGeminiAgent(client=client, name="Closure Agent", instructions=closure_instructions)

        routine_instructions = ["You are a recovery routine planner that:", "1. Designs 7-day recovery challenges", "2. Includes fun activities and self-care tasks", "3. Suggests social media detox strategies", "4. Creates empowering playlists", "Focus on practical recovery steps"]
        routine_planner_agent = SimpleGeminiAgent(client=client, name="Routine Planner Agent", instructions=routine_instructions)

        honesty_instructions = ["You are a direct feedback specialist that:", "1. Gives raw, objective feedback about breakups", "2. Explains relationship failures clearly", "3. Uses blunt, factual language", "4. Provides reasons to move forward", "Focus on honest insights without sugar-coating"]
        brutal_honesty_agent = SimpleGeminiAgent(client=client, name="Brutal Honesty Agent", instructions=honesty_instructions)

        return therapist_agent, closure_agent, routine_planner_agent, brutal_honesty_agent
    except APIError as e:
        st.error(f"Error de API: {str(e)}. Por favor, verifica tu clave API y el estado de la Gemini API en tu proyecto.")
        return None, None, None, None
    except Exception as e:
        st.error(f"Error al inicializar: {str(e)}")
        return None, None, None, None


# --- Construcción de la Interfaz de Usuario (UI) con Streamlit ---

st.set_page_config(page_title="💔 Breakup Recovery Squad", page_icon="💔", layout="wide")

with st.sidebar:
    st.header("🔑 API Configuration")
    if "api_key_input" not in st.session_state: st.session_state.api_key_input = ""
    api_key = st.text_input(
        "Enter your Gemini API Key", value=st.session_state.api_key_input,
        type="password", help="Get your API key from Google AI Studio", key="api_key_widget"
    )
    if api_key != st.session_state.api_key_input: st.session_state.api_key_input = api_key
    if api_key: st.success("API Key provided! ✅")
    else: st.warning("Please enter your API key to proceed")

st.title("💔 Breakup Recovery Squad")
st.markdown("### Your AI-powered breakup recovery team is here to help!")

col1, col2 = st.columns(2)
with col1:
    st.subheader("Share Your Feelings")
    user_input_text = st.text_area(
        "How are you feeling? What happened?", height=150, placeholder="Tell us your story..."
    )
with col2:
    st.subheader("Upload Chat Screenshots")
    uploaded_files = st.file_uploader(
        "Upload screenshots of your chats (optional)", type=["jpg", "jpeg", "png"],
        accept_multiple_files=True, key="screenshots"
    )

if st.button("Get Recovery Plan 💝", type="primary"):
    if not st.session_state.api_key_input:
        st.warning("Please enter your API key in the sidebar first!")
        st.stop()

    # NUEVO GUARDRALL: BLOQUEO DE ENTRADA
    if user_input_text:
        is_safe, processed_input = check_and_clean_input(user_input_text)

        # Si NO es seguro (contiene PII o groserías), mostramos advertencia y paramos la ejecución
        if not is_safe:
            # Checkamos si la razón es PII o lenguaje
            contains_pii = any(re.search(pattern, user_input_text) for pattern in PII_KEYWORDS)
            contains_swears = any(word in user_input_text.lower() for word in PROHIBITED_LANGUAGE)

            warning_message = "⚠️ **Advertencia de Seguridad:** "

            if contains_pii and contains_swears:
                 warning_message += "El sistema detectó **datos personales** y **lenguaje inapropiado**. Por tu seguridad, no podemos procesar esta solicitud. Por favor, reformula tu mensaje sin compartir información sensible o usar palabras groseras."
            elif contains_pii:
                 warning_message += "El sistema detectó **datos personales** (ej. IDs, números) en tu mensaje. Por tu seguridad y privacidad, **no podemos procesar esta solicitud**. Por favor, reformula tu mensaje sin compartir información sensible."
            elif contains_swears:
                 warning_message += "El sistema detectó **lenguaje inapropiado**. Por favor, mantén un lenguaje respetuoso para continuar con el plan de recuperación."

            st.error(warning_message)
            st.stop() # Detiene la ejecución antes de llamar a los agentes

        # Si es seguro y limpio, procedemos a inicializar los agentes
        user_input_text = processed_input # Usamos la versión limpia

        agents = initialize_agents(st.session_state.api_key_input)
        if all(agents):
            therapist_agent, closure_agent, routine_planner_agent, brutal_honesty_agent = agents

            try:
                st.header("Su plan de recuperación personalizado")

                # ... (Llamadas a agentes con el user_input_text limpio)

                # 1. Llamada al Agente Terapeuta.
                with st.spinner("🤗 Obteniendo apoyo empático..."):
                    therapist_prompt = f"Analiza el estado emocional basado en: Mensaje del usuario: {user_input_text}"
                    response = therapist_agent.run(input=therapist_prompt)
                    st.subheader("🤗 Apoyo emocional")
                    st.markdown(response.content)

                time.sleep(5)

                # 2. Llamada al Agente de Cierre.
                with st.spinner("✍️ Creando mensajes de cierre..."):
                    closure_prompt = f"Ayuda a crear cierre emocional basado en: Sentimientos del usuario: {user_input_text}"
                    response = closure_agent.run(input=closure_prompt)
                    st.subheader("✍️ Encontrando Cierre")
                    st.markdown(response.content)

                time.sleep(5)

                # 3. Llamada al Agente Planificador de Rutinas.
                with st.spinner("📅 Creando tu plan de recuperación..."):
                    routine_prompt = f"Diseña un plan de recuperación de 7 días basado en: Estado actual: {user_input_text}"
                    response = routine_planner_agent.run(input=routine_prompt)
                    st.subheader("📅 Tu Plan de Recuperación")
                    st.markdown(response.content)

                time.sleep(5)

                # 4. Llamada al Agente de Honestidad Brutal.
                with st.spinner("💪 Obteniendo perspectiva honesta..."):
                    honesty_prompt = f"Proporciona retroalimentación honesta y constructiva sobre: Situación: {user_input_text}"
                    response = brutal_honesty_agent.run(input=honesty_prompt)
                    st.subheader("💪 Perspectiva Honesta")
                    st.markdown(response.content)
            except Exception as e:
                st.error(f"Ocurrió un error durante el análisis: {e}")
        else:
            st.error("Fallo al inicializar los agentes. Por favor, verifica tu clave API.")
    else:
        st.warning("Por favor, comparte tus sentimientos para obtener ayuda.")

st.markdown("---")
st.markdown("<div style='text-align: center'><p>Hecho con ❤️ por el Equipo de Recuperación de Rupturas</p></div>", unsafe_allow_html=True)

Overwriting app.py


In [3]:
# Celda 3: Ejecución de Streamlit y ngrok
from pyngrok import ngrok
import os

# ⚠️ Asegúrate de que este sea tu token de autenticación de ngrok
NGROK_AUTH_TOKEN = "32eAJto4Iq2t5rCDOa2Wjcvso39_3Pdp7b48yUjgtaLaUNMPQ"

# 1. Autentica tu cuenta de ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 2. Termina CUALQUIER túnel ngrok existente
ngrok.kill()

try:
    os.system("pkill -9 streamlit")

    # 3. Abre un túnel público
    public_url = ngrok.connect(8501)
    print(f"✅ Tu aplicación está lista. Haz clic en el siguiente enlace para abrirla:\n{public_url}")

    # 4. Ejecuta el archivo app.py de Streamlit en segundo plano
    os.system("streamlit run app.py &")

except Exception as e:
    print(f"❌ Ocurrió un error al iniciar ngrok: {e}")

✅ Tu aplicación está lista. Haz clic en el siguiente enlace para abrirla:
NgrokTunnel: "https://07da5fa9d65d.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# NUEVA Celda 4: Iniciar la aplicación de Streamlit
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.245.80:8501

